In [7]:
#Install Packages
#!pip install langchain pip install langchain_community langchain-openai openai sqlalchemy

In [66]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, inspect, text

In [68]:
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain_openai import ChatOpenAI
import openai

In [69]:
#Load CSVs to Pandas Dataframes
customers_df = pd.read_csv('text_to_sql_bot/customers_data.csv')
payments_df = pd.read_csv('text_to_sql_bot/payments_data.csv')

#Create an SQLAlchemy engine for SQLite
#https://www.datacamp.com/tutorial/sqlalchemy-tutorial-examples
engine = create_engine('sqlite:///example.db')

# Write the DataFrames to the SQL database
customers_df.to_sql('customers', engine, if_exists='replace', index=False)
payments_df.to_sql('payments', engine, if_exists='replace', index=False)

500

In [70]:
# Test Database Connection
with engine.connect() as connection:
    # Use SQLAlchemy inspector to list tables
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    print("Tables in the database:", tables)
    
     # Query each table and display a sample of data
    for table in tables:
        print(f"\nSample data from {table}:")
        query = text(f"SELECT * FROM {table} LIMIT 5;")
        result = connection.execute(query)
        for row in result:
            print(row)

Tables in the database: ['customers', 'payments']

Sample data from customers:
(804167, '2022-08-12', 'NH')
(909833, '2022-11-01', 'AR')
(407014, '2023-04-06', 'NY')
(564697, '2022-01-28', 'ME')
(548060, '2023-11-21', 'IA')

Sample data from payments:
('c52fffa5-15bd-4b3a-9e72-fbfa78e754f5', 837542, 950447, '2023-12-21', 'FAILURE', 77188)
('2b17845b-87b4-4254-b90b-fc95017e2c9e', 190186, 576925, '2023-06-12', 'SUCCESS', 75299)
('3634b895-b9ed-4e82-be63-860038de20e3', 508607, 548060, '2023-12-04', 'FAILURE', 22181)
('e4f4d48b-833c-4643-8df8-798553eb8b7c', 335083, 714482, '2023-11-29', 'FAILURE', 14573)
('bcb333a7-18c3-40b3-8baa-e01ba1bb4da0', 558046, 647995, '2023-12-17', 'FAILURE', 96635)


In [72]:
# Step 2: Initialize the OpenAI LLM with API key
llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo-0125", openai_api_key= 'your_open_ai_key')

# Step 3: Initialize the SQLDatabase object with the SQLAlchemy engine
db = SQLDatabase(engine)

# Create LangChain SQL Agent 
#https://python.langchain.com/v0.1/docs/use_cases/sql/agents/
sql_agent = create_sql_agent(llm=llm, db=db, verbose=True, agent_type="zero-shot-react-description")

In [73]:
# Step 4: Ask a question in natural language
question = "Find the total Number of Unique Transactions"
response = sql_agent.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, paymentsI should query the schema of the payments table to see if it contains information about transactions.
Action: sql_db_schema
Action Input: payments
CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	22181
*/I can use the payments table to find the total number of unique transactions by counting the distinct payment_id values.
Action: sql_db_query
Action Input: SELECT COUNT(DISTINCT payment_id) AS total_unique_transactions FROM payments[(500,)]I now know the final ans

In [113]:
#Use Callbacks to get the final SQL Query as well as steps taken by the agent.
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction

class SQLHandler(BaseCallbackHandler):
    def __init__(self):
        self.sql_result = None
        self.sql_result_log = []
        
    def on_agent_action(self, action: AgentAction, **kwargs):
        #print(f"Action Tool: {action.tool}")
        """Run on agent action. If the tool being used is sql_db_query,
        it means we're submitting the SQL and we can record it as the final SQL."""
        if action.tool == "sql_db_query":
            self.sql_result = action.tool_input
            #print(f"SQL Query Captured: {self.sql_result}")
        self.sql_result_log.append(action.log)
# Instantiate your SQLHandler
sql_handler = SQLHandler()

In [114]:
# Invoke the SQL agent with the custom callback handler
question = "Find the total Number of Unique Transactions"
result = sql_agent.run(question, callbacks=[sql_handler])




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, paymentsI should query the payments table to find unique transactions.
Action: sql_db_schema
Action Input: payments
CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	22181
*/I can use the payments table to find the total number of unique transactions by counting the distinct payment_id values.
Action: sql_db_query
Action Input: SELECT COUNT(DISTINCT payment_id) AS total_unique_transactions FROM payments[(500,)]I now know the final answer
Final Answer: The total number of u

In [160]:
# Access the result
print("Result:", result)
print("")
# Access the SQL query
print("Generated SQL:", sql_handler.sql_result)
print("")

print("Steps Taken By Agent:")
for (line,i) in zip(sql_handler.sql_result_log,range(1,len(sql_handler.sql_result_log)+1)):
    print("Step -", i, ":\n", line, sep="")
    print("\n")

Result: The total number of unique transactions is 500.

Generated SQL: SELECT COUNT(DISTINCT payment_id) AS total_unique_transactions FROM payments

Steps Taken By Agent:
Step -1:
Action: sql_db_list_tables
Action Input: 


Step -2:
I should query the payments table to find unique transactions.
Action: sql_db_schema
Action Input: payments


Step -3:
I can use the payments table to find the total number of unique transactions by counting the distinct payment_id values.
Action: sql_db_query
Action Input: SELECT COUNT(DISTINCT payment_id) AS total_unique_transactions FROM payments




In [56]:
#Q1
question = "Find the Total number of Unique Transactions, Senders and Recipients with Successful transactions ?"
response = sql_agent.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""customers, paymentsThe payments table seems to be the most relevant for this question. I should check its schema to understand its structure.
Action: sql_db_schema
Action Input: "payments"
CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	22181
*/The payments table has the columns I need to answer the question. I need to count the unique payment_id for transactions, sender_id for senders, and recipient_id for recipients. But I only need to consider the rows where payment_state is '

In [64]:
#Q2
question = "Find the failure rate of transactions by sender’s geographical state and return the top 10 states with the highest failure rate?"
response = sql_agent.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""customers, paymentsThe tables 'customers' and 'payments' seem relevant to the question. I should check their schemas to see if they contain the necessary information.
Action: sql_db_schema
Action Input: "customers, payments"
CREATE TABLE customers (
	customer_id BIGINT, 
	created_at TEXT, 
	state TEXT
)

/*
3 rows from customers table:
customer_id	created_at	state
804167	2022-08-12	NH
909833	2022-11-01	AR
407014	2023-04-06	NY
*/


CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	2

In [59]:
#Q3
question = "Find the Customer or Customers that sent or received the most amount in the year 2022 irrespective of if the transactio was a success or failure?"
response = sql_agent.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""customers, paymentsThe tables 'customers' and 'payments' seem relevant to the question. I should check their schemas to understand the structure of the data.
Action: sql_db_schema
Action Input: "customers, payments"
CREATE TABLE customers (
	customer_id BIGINT, 
	created_at TEXT, 
	state TEXT
)

/*
3 rows from customers table:
customer_id	created_at	state
804167	2022-08-12	NH
909833	2022-11-01	AR
407014	2023-04-06	NY
*/


CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	22181
*/Th

In [60]:
#Q4
question = "Find all Customers that opened an account in August 2022 that have sent or received >$100 in the first 30 days of opening the account?"
response = sql_agent.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""customers, paymentsThe tables 'customers' and 'payments' seem relevant to the question. I should check their schemas to understand the structure and fields.
Action: sql_db_schema
Action Input: "customers, payments"
CREATE TABLE customers (
	customer_id BIGINT, 
	created_at TEXT, 
	state TEXT
)

/*
3 rows from customers table:
customer_id	created_at	state
804167	2022-08-12	NH
909833	2022-11-01	AR
407014	2023-04-06	NY
*/


CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	22181
*/The

In [61]:
#Q5
question = "Find pairs of customers who successfully sent and received money between each other?"
response = sql_agent.run(question)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""customers, paymentsThe tables 'customers' and 'payments' seem relevant to the question. I should check their schemas to understand the structure of the data.
Action: sql_db_schema
Action Input: "customers, payments"
CREATE TABLE customers (
	customer_id BIGINT, 
	created_at TEXT, 
	state TEXT
)

/*
3 rows from customers table:
customer_id	created_at	state
804167	2022-08-12	NH
909833	2022-11-01	AR
407014	2023-04-06	NY
*/


CREATE TABLE payments (
	payment_id TEXT, 
	sender_id BIGINT, 
	recipient_id BIGINT, 
	created_at TEXT, 
	payment_state TEXT, 
	amount_cents BIGINT
)

/*
3 rows from payments table:
payment_id	sender_id	recipient_id	created_at	payment_state	amount_cents
c52fffa5-15bd-4b3a-9e72-fbfa78e754f5	837542	950447	2023-12-21	FAILURE	77188
2b17845b-87b4-4254-b90b-fc95017e2c9e	190186	576925	2023-06-12	SUCCESS	75299
3634b895-b9ed-4e82-be63-860038de20e3	508607	548060	2023-12-04	FAILURE	22181
*/Th